# Notebook 2: All Parks
Now that we established our method for MacArthur Park, the next step was to apply this process to 12 parks across LA we selected based on a quick Flickr search to determine if there were a variety of photos with a variety of topics. 

This notebook is broken into three sections:
1. Inspect & Explore the Flickr API
2. Data Wrangling
3. Visualizing Cultural Ecosystem Services

In [1]:
#The first step was to once again call the API function

import flickrapi
import json

api_key = u'f950122b83b682c546201f10d33edffe'
api_secret = u'057c65cd7fe1b2c8'

#flickr = flickrapi.FlickrAPI(api_key, api_secret)
#for json format
flickr = flickrapi.FlickrAPI(api_key, api_secret, format='parsed-json')

This time we wanted to pull data from all 12 pre-selected parks. Here, we searched by tags identifying each park, then saved into a dataframe 

In [2]:
import pandas as pd

def get_parks(num_pages):
    park_list = []
    for i in range(1, num_pages+1): #range documentation starts at 0, +1 ensures we pull the page number we feed the function below
        extras = 'geo,description,tags'
        tags = ['MacArthur Park, Woodley Avenue Park, Rio de Los Angeles State Park, Runyon Canyon, Temescal Gateway, Heidelberg Park, Hancock Park, Franklin Canyon Park, Angels Gate, Coldwater Canyon, Chatsworth Park South, Cheviot Hills, O''Melveny Park']
        parks_LA = flickr.photos.search(tags=tags, bbox = '-118.898278,33.704902,-118.161021,34.32848',
                                        method_name='flickr', page=i, per_page=500, extras=extras)  
        
        #.extend combines each page of search results together: https://www.programiz.com/python-programming/methods/list/extend
        park_list.extend(parks_LA['photos']['photo']) #pulls data from each individual photo 
        
    #reorients the data and converts to pandas df: https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe
    df = pd.DataFrame.from_dict(park_list, orient='columns') 
    
    return df

parks_data = get_parks(12) #pulls data from all 12 pages of photos

In [3]:
#checking the dataframe 
parks_data

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,description,...,latitude,longitude,accuracy,context,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family
0,49856813066,9771767@N04,83838b7d7b,65535,66,Angel's Gate Park,1,0,0,"{'_content': 'Have a wonderful day, everyone...'}",...,33.709932,-118.293882,16,0,,5392528,1,0,0,0
1,51050150612,192454804@N08,d07257b1b3,65535,66,English Home,1,0,0,{'_content': 'English style house in Los Angel...,...,34.074580,-118.334275,16,0,,5355165,1,0,0,0
2,50952829406,77318907@N08,6fe5b8ef3a,65535,66,Angel's Gate Cloudscape,1,0,0,"{'_content': 'San Pedro, CA 01-02-21'}",...,33.721867,-118.271759,16,0,,5392528,1,0,0,0
3,50952896937,77318907@N08,ebe224b984,65535,66,Harbor Entrance at Sunrise,1,0,0,"{'_content': 'The Angel's Gate in San Pedro, C...",...,33.721707,-118.271791,16,0,,5392528,1,0,0,0
4,50474879407,54718757@N00,a011599de5,65535,66,"22,834",1,0,0,"{'_content': '22,834 smoke on the water, in th...",...,33.732833,-118.317989,16,0,,5392544,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,280833391,67833269@N00,b14241e30a,102,1,Jon Nakamura 10-27-06,1,0,0,{'_content': 'Angels Gate Cultural Center Stud...,...,33.713961,-118.294783,16,0,.D8LiGdQUL_LN4aBcw,12587688,1,0,0,0
2746,279079416,63348143@N00,4b0eb50c34,103,1,cleavage,1,0,0,{'_content': ''},...,34.081597,-118.327217,16,0,efoucK5TWr0U1ThAlQ,28751566,1,0,0,0
2747,279078460,63348143@N00,a8a2f65a8e,90,1,cement,1,0,0,{'_content': ''},...,34.081597,-118.327217,16,0,efoucK5TWr0U1ThAlQ,28751566,1,0,0,0
2748,277276618,36521983488@N01,a7a3b3b36f,115,1,,1,0,0,{'_content': 'got a ton of pics to upload from...,...,34.108437,-118.213984,16,0,7Z5HMmpTVr4VzDpD,2442047,1,0,0,0


The next step was to create a new column and assign the park name to that column so that we could more easily track tags as they relate to specific parks.

In [4]:
# Create a new column and assign the park name 
park_names = ['macarthur', 'woodley', 'riodelosangeles', 'runyoncanyon', 'temescalgateway', 'heidelbergpark', 'hancockpark', 'franklincanyonpark', 'angelsgate', 'coldwatercanyon', 'chatsworthparksouth','cheviothills']

def get_park_name(row):
    for park in park_names:
        if park in row['tags']:
            return park
    return 'Unknown'

parks_data['parkname'] = parks_data.apply(lambda row: get_park_name(row), axis=1)

In [5]:
#checking our work
parks_data

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,description,...,longitude,accuracy,context,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family,parkname
0,49856813066,9771767@N04,83838b7d7b,65535,66,Angel's Gate Park,1,0,0,"{'_content': 'Have a wonderful day, everyone...'}",...,-118.293882,16,0,,5392528,1,0,0,0,angelsgate
1,51050150612,192454804@N08,d07257b1b3,65535,66,English Home,1,0,0,{'_content': 'English style house in Los Angel...,...,-118.334275,16,0,,5355165,1,0,0,0,hancockpark
2,50952829406,77318907@N08,6fe5b8ef3a,65535,66,Angel's Gate Cloudscape,1,0,0,"{'_content': 'San Pedro, CA 01-02-21'}",...,-118.271759,16,0,,5392528,1,0,0,0,angelsgate
3,50952896937,77318907@N08,ebe224b984,65535,66,Harbor Entrance at Sunrise,1,0,0,"{'_content': 'The Angel's Gate in San Pedro, C...",...,-118.271791,16,0,,5392528,1,0,0,0,angelsgate
4,50474879407,54718757@N00,a011599de5,65535,66,"22,834",1,0,0,"{'_content': '22,834 smoke on the water, in th...",...,-118.317989,16,0,,5392544,1,0,0,0,angelsgate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2745,280833391,67833269@N00,b14241e30a,102,1,Jon Nakamura 10-27-06,1,0,0,{'_content': 'Angels Gate Cultural Center Stud...,...,-118.294783,16,0,.D8LiGdQUL_LN4aBcw,12587688,1,0,0,0,angelsgate
2746,279079416,63348143@N00,4b0eb50c34,103,1,cleavage,1,0,0,{'_content': ''},...,-118.327217,16,0,efoucK5TWr0U1ThAlQ,28751566,1,0,0,0,hancockpark
2747,279078460,63348143@N00,a8a2f65a8e,90,1,cement,1,0,0,{'_content': ''},...,-118.327217,16,0,efoucK5TWr0U1ThAlQ,28751566,1,0,0,0,hancockpark
2748,277276618,36521983488@N01,a7a3b3b36f,115,1,,1,0,0,{'_content': 'got a ton of pics to upload from...,...,-118.213984,16,0,7Z5HMmpTVr4VzDpD,2442047,1,0,0,0,heidelbergpark


In [6]:
#save to a CSV file so we do not have to keep pulling from Flickr

parks_data.to_csv('parks_data.csv', index=False)  

Next we take a look at the tags to determine how best to clean them up. 

***I think we can create a tags only df here from the .csv and use that from here on out

In [7]:
#importing parks csv
import pandas as pd

# You might need to add a path as well
fn = 'parks_data.csv'
parks_data = pd.read_csv(fn)
parks_data.head()

,id,owner,secret,server,farm,title,ispublic,isfriend,isfamily,description,...,longitude,accuracy,context,place_id,woeid,geo_is_public,geo_is_contact,geo_is_friend,geo_is_family,parkname
0,49856813066,9771767@N04,83838b7d7b,65535,66,Angel's Gate Park,1,0,0,"{'_content': 'Have a wonderful day, everyone...'}",...,-118.293882,16,0,NaN,5392528.0,1,0,0,0,angelsgate
1,51050150612,192454804@N08,d07257b1b3,65535,66,English Home,1,0,0,{'_content': 'English style house in Los Angel...,...,-118.334275,16,0,NaN,5355165.0,1,0,0,0,hancockpark
2,50952829406,77318907@N08,6fe5b8ef3a,65535,66,Angel's Gate Cloudscape,1,0,0,"{'_content': 'San Pedro, CA\n01-02-21'}",...,-118.271759,16,0,NaN,5392528.0,1,0,0,0,angelsgate
3,50952896937,77318907@N08,ebe224b984,65535,66,Harbor Entrance at Sunrise,1,0,0,"{'_content': ""The Angel's Gate in San Pedro, C...",...,-118.271791,16,0,NaN,5392528.0,1,0,0,0,angelsgate
4,50474879407,54718757@N00,a011599de5,65535,66,"22,834",1,0,0,"{'_content': '22,834 smoke on the water, in th...",...,-118.317989,16,0,NaN,5392544.0,1,0,0,0,angelsgate


In [8]:
#create function to remove stop words, tokenize, convert all text to lowercase, and convert string to list
#We also went back and created a list from our tags of common words we did not want to analyze (i.e. losangeles, la, california, etc)

import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

swords = [re.sub(r"[^A-z\s]", "", sword) for sword in stopwords.words('english')]
swords += ['losangeles', 'la', 'losangelesca', 'ca', 'dtla', 'macarthur', 'macarthurpark', 'woodley', 'riodelosangeles', 'runyoncanyon', 
           'temescalgateway', 'heidelbergpark', 'hancockpark', 'franklincanyonpark', 'franklincanyonpark', 'angelsgate', 
           'coldwatercanyon', 'chatsworthparksouth','cheviothills', 'california', 'usa', 'southerncalifornia', 'park', 'parklabrea', 
          'unitedstates', 'america']

def clean_string(text):
    # remove punctuation
    text = re.sub(r"[^A-z\s]", "", text)
    
    cleaned_list_of_words = [word for word in word_tokenize(text.lower()) if word not in swords] #return a string or apply to all tags
    
    return cleaned_list_of_words

#calling the function to only apply to the tags column 
parks_data['tags'] = parks_data['tags'].apply(clean_string)

In [9]:
# Check to make sure tags are in a list
parks_data['tags']

0       [koreanfriendshipbell, korean, friendshipbell,...
1                             [englishhome, architecture]
2       [losangelesharbor, sanpedro, sunrise, colorful...
3                                      [sanpedro, harbor]
4       [jrglongbeach, smoke, water, harbor, hiking, s...
                              ...                        
2745    [jon, artist, studioartist, agcc, angelsgatecu...
2746    [camera, cars, corner, clinton, rudy, tiles, h...
2747    [morning, corner, truck, clinton, cement, holl...
2748    [auto, car, vw, volkswagen, automobile, conver...
2749    [intern, agcc, angelsgateculturalcenter, mindy...
Name: tags, Length: 2750, dtype: object

For our next step, we needed to break apart the tags so tha no more than one tag is assocated with each park name. To do this, we used the .explode function. When moved each list item (the tags) to a new row. 

In [10]:
#extend the daraframe by pairing up each individual tag by park name
#Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.explode.html

cols = ['tags', 'parkname']
tag_park = parks_data[cols].explode('tags', ignore_index=True)

Next, we grouped the tags by park name to determine how many tags are associated with each park

In [11]:
#getting a simple count of tags per park 
print(tag_park.groupby('parkname').count())

                      tags
parkname                  
Unknown                126
angelsgate            4021
chatsworthparksouth     12
cheviothills           350
coldwatercanyon        213
franklincanyonpark     319
hancockpark          12242
heidelbergpark          11
macarthur             3812
riodelosangeles        151
runyoncanyon          2433
temescalgateway          3
woodley                 39


Next, we obtained the top 100 most-used tags across all parks.

In [12]:
#groupby documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html
#sort_values documentation: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

#create a new column, value, using a list comprehension, and I'll temporarily fill each row with a value of 1
#I added this code because I intially tried to run the below code, and was not getting a separate "value" count column so I added this as a temporary fill
tag_park['value'] = [1 for i in range(tag_park.shape[0])]

#return top 100 most used tags sorted by value
top_100_tags = tag_park.groupby('tags').sum().sort_values('value', ascending=False).head(100)

#so we can view all tags
pd.set_option('display.max_rows', 100)

print(top_100_tags)

                                         value
tags                                          
tarpits                                    562
labreatarpits                              443
museum                                     355
labrea                                     343
agcc                                       342
art                                        251
angelsgateculturalcenter                   241
socal                                      241
fossils                                    218
openstudios                                208
bones                                      195
hollywood                                  193
pagemuseum                                 191
westlake                                   185
paleontology                               177
lacma                                      167
animals                                    146
realestate                                 144
estate                                     144
real         

In [13]:
# exporting top 100 tags to a csv for hand coding 
top_100_tags.to_csv('top_100_tags.csv', index=True)

In [14]:
# pulling top 200 tags 

#create a new column, value, using a list comprehension, and I'll temporarily fill each row with a value of 1
#I added this code because I intially tried to run the below code, and was not getting a separate "value" count column so I added this as a temporary fill
tag_park['value'] = [1 for i in range(tag_park.shape[0])]

#return top 100 most used tags sorted by value
top_200_tags = tag_park.groupby('tags').sum().sort_values('value', ascending=False).head(200)

#so we can view all tags
pd.set_option('display.max_rows', 200)

print(top_200_tags)

                                         value
tags                                          
tarpits                                    562
labreatarpits                              443
museum                                     355
labrea                                     343
agcc                                       342
art                                        251
angelsgateculturalcenter                   241
socal                                      241
fossils                                    218
openstudios                                208
bones                                      195
hollywood                                  193
pagemuseum                                 191
westlake                                   185
paleontology                               177
lacma                                      167
animals                                    146
realestate                                 144
estate                                     144
real         

In [15]:
# exporting top 200 tags to a csv for hand coding 
top_200_tags.to_csv('top_200_tags.csv', index=True)

Finally, we want to obtain the top 50 tags by park 

In [18]:
#create a new dataframe, grouping by tags and parkname, and then find the sum of each tags' occurence 
tag_park_counts = tag_park.groupby(cols).sum()

#we want unique tags associated with each individual park name, so we use the .unique function: https://www.w3resource.com/numpy/manipulation/unique.php
park_names = tag_park['parkname'].unique()

#then write a for loop to loop through each park name, returning the top 50 most common tags for each park, sorted by the "value" column (i.e. the frequency of that tags' occurence)
#.index.get_level_values allows us to sort by park name: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.get_level_values.html
park_counts = {}
for park in park_names:
    park_counts[park] = tag_park_counts.iloc[tag_park_counts.index.get_level_values('parkname') == park].sort_values('value', ascending=False).head(100)

park_counts

{'angelsgate':                                                        value
 tags                                       parkname         
 agcc                                       angelsgate    342
 angelsgateculturalcenter                   angelsgate    241
 openstudios                                angelsgate    208
 art                                        angelsgate    180
 artonthewaterfront                         angelsgate    142
 allankaprow                                angelsgate     81
 happening                                  angelsgate     81
 portoflosangeles                           angelsgate     80
 ice                                        angelsgate     76
 fluids                                     angelsgate     73
 gallerya                                   angelsgate     71
 angelsgatepark                             angelsgate     71
 pola                                       angelsgate     70
 iceblocks                                  angelsgate  

In [17]:
park_counts.to_csv('park_counts.csv', index=False) 

AttributeError: 'dict' object has no attribute 'to_csv'

In [ ]:
### ADAPT using the new tags only DF

#Use the function to clean up the list of tags and return a new list of cleaned tags
cleaned_macarthur_tags = [clean_string(x) for x in macarthurDf_tags]
print(cleaned_macarthur_tags)